Ön hazırlık (kütüphane kurulumu)

In [1]:
# Gerekli kütüphaneleri yükleyelim (Colab hücresine)
!pip install cirq qiskit qutip numpy matplotlib --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.8/670.8 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 430.5/430.5 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 102.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.8/31.8 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 70.3 MB/s eta 0:00:00


Cirq ile Teleportasyon Devresi Örneği

In [2]:
import cirq

# Qubitler
q_psi = cirq.LineQubit(0)   # Gönderilecek bilinmeyen durum
q_alice = cirq.LineQubit(1)
q_bob = cirq.LineQubit(2)

circuit = cirq.Circuit()

# Başlangıç: Bell durumu Alice-Bob
circuit.append([cirq.H(q_alice), cirq.CNOT(q_alice, q_bob)])

# Alice'in gönderilecek bilinmeyen durumu (örnek: |ψ> = |0> + |1>)/√2
circuit.append(cirq.H(q_psi))

# Alice CNOT ve Hadamard
circuit.append(cirq.CNOT(q_psi, q_alice))
circuit.append(cirq.H(q_psi))

# Alice ölçümü
circuit.append([cirq.measure(q_psi, key='m0'), cirq.measure(q_alice, key='m1')])

# Bob’un düzeltmesi (klasik bitlere bağlı)
# Simülasyonda conditional uygulama
# Not: Cirq'de klasik bitlere bağlı gate conditional olarak eklenebilir
# Burada sadece devre yapısı gösterildi

print(circuit)


0: ───H───────@───H─────────M('m0')───
              │
1: ───H───@───X───M('m1')─────────────
          │
2: ───────X───────────────────────────


Qiskit ile Teleportasyon

In [3]:
from qiskit import QuantumCircuit, Aer, execute
from qiskit.visualization import plot_bloch_multivector, plot_histogram
from qiskit.quantum_info import Statevector

# Qubitler: 0 -> psi, 1 -> Alice, 2 -> Bob
qc = QuantumCircuit(3, 2)

# Adım 1: Bell durumu oluştur
qc.h(1)
qc.cx(1, 2)

# Adım 2: Gönderilecek qubit (örnek: |ψ> = (|0> + |1>)/√2)
qc.h(0)

# Adım 3: Alice CNOT ve Hadamard
qc.cx(0, 1)
qc.h(0)

# Adım 4: Alice ölçümü
qc.measure([0,1], [0,1])

# Devreyi çiz
qc.draw('mpl')


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

In [4]:
# Simülasyon ve Bob’un Durumu

from qiskit.providers.aer import AerSimulator
sim = AerSimulator()

result = sim.run(qc).result()
counts = result.get_counts()
print("Alice ölçüm sonuçları:", counts)

ModuleNotFoundError: No module named 'qiskit.providers.aer'

In [5]:
qc = QuantumCircuit(3, 2)
qc.h(1)
qc.cx(1,2)
qc.h(0)
qc.cx(0,1)
qc.h(0)
qc.measure([0,1],[0,1])
qc.x(2).c_if(qc.cregs[0], 1)  # Alice 01 ölçerse Bob X uygular
qc.z(2).c_if(qc.cregs[0], 2)  # Alice 10 ölçerse Bob Z uygular
qc.z(2).x(2).c_if(qc.cregs[0], 3)  # Alice 11 ölçerse Bob ZX uygular


AttributeError: 'InstructionSet' object has no attribute 'c_if'

QuTiP ile Teleportasyon (Analitik)

In [6]:
import qutip as qt

# Qubitler: psi (0), Alice (1), Bob (2)
# |psi> = (|0> + |1>)/sqrt(2)
psi = (qt.basis(2,0) + qt.basis(2,1)).unit()

# Bell durumu |B00> Alice-Bob
B00 = (qt.tensor(qt.basis(2,0), qt.basis(2,0)) + qt.tensor(qt.basis(2,1), qt.basis(2,1))).unit()

# Tam sistem: psi ⊗ B00
state = qt.tensor(psi, B00)

# Alice CNOT (0 kontrol, 1 hedef)
CNOT_01 = qt.cnot(3, control=0, target=1)
state = CNOT_01 * state

# Alice Hadamard (0. qubit)
H0 = qt.tensor(qt.hadamard_transform(), qt.qeye(2), qt.qeye(2))
state = H0 * state

# Ölçüm ve Bob'un durumu
# Ölçüm için QuTiP'de proje olarak alabiliriz
m00 = qt.tensor(qt.basis(2,0)*qt.basis(2,0).dag(), qt.qeye(2), qt.qeye(2))
m01 = qt.tensor(qt.basis(2,0)*qt.basis(2,0).dag(), qt.basis(2,1)*qt.basis(2,1).dag(), qt.qeye(2))
# Diğer ölçümler benzer şekilde

print("Tam sistem durumu:\n", state)


AttributeError: module 'qutip' has no attribute 'cnot'